In [ ]:
from cv2 import cv2
import numpy as np
import os

# Playing video from file:
cap = cv2.VideoCapture('input.mp4') # change this to video in question 

try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print ('Error: Creating directory of data')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Saves image of the current frame in jpg file
    name = './data/' + str(int(currentFrame)) + '.jpg'
    print ('Creating...' + name)
    cv2.imwrite(name, frame)

    # To stop duplicate images
    currentFrame += 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [14]:
import requests
import json

predictionUrl = 'https://myazurecustomvision.cognitiveservices.azure.com/customvision/v3.0/Prediction/fd698d74-978e-440e-b34f-a802c01f5d87/detect/iterations/Iteration3/image'
predictionKey = '9a1d0210845f417c8728965579867416'

def predict(image):
  headers = {'content-type' : 'application/octet-stream','Prediction-Key' : predictionKey}
  request = requests.post(predictionUrl,data=open(image,"rb"),headers=headers)

  probability = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['probability']
  tag = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['tagName']
  box = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['boundingBox']

  return probability, tag, box
  

In [15]:
import time

myList = []

for i in range(0, len(os.listdir('data')) - 1):
  myList.append(predict("./data/" + str(i) + ".jpg"))
  time.sleep(0.5)

In [16]:
def drawBox(boxDetails, image, i):
  if(boxDetails[0] < 0.7):
    return

  x = boxDetails[2]['left']
  y = boxDetails[2]['top']
  w = boxDetails[2]['width']
  h = boxDetails[2]['height']
  
  width, height, channels = image.shape

  startX = int(x * height)
  startY = int(y * width)
  endX = int(w * height) + startX
  endY = int(h * width) + startY

  cv2.imwrite("data/%d.jpg" % i, cv2.rectangle(image, (startX, startY), (endX, endY), (155, 255, 0), 2))

In [17]:
for i in range(0, len(os.listdir('data')) - 1):
  image = cv2.imread("./data/" + str(i) + ".jpg")
  drawBox(myList[i], image, i)

In [21]:
#Reassemble drawn images here into video
images = []

for i in range(0, len(os.listdir('data')) - 1):
  images.append("data/%d.jpg" % i)

frame = cv2.imread('data/0.jpg')
height, width, layers = frame.shape

video = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (width,height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()